In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import pandas as pd
import albumentations as A
from albumentations import *
from albumentations.pytorch.transforms import ToTensorV2
import effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, create_model_from_config
from effdet.efficientdet import HeadNet

from datetime import datetime
import time
import random
import cv2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import GradScaler, autocast
from glob import glob

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(0)

In [4]:
train_ = pd.read_csv('data/labels_train.csv')

labelmap = {'fruit_healthy':1, 'fruit_woodiness':2, 'fruit_brownspot':3}
labelmap_inv = {k:v for v,k in labelmap.items()}

class PassionFruitDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, transforms):
        self.csv = pd.read_csv(csv_path)
        self.transforms = transforms
        self.mode = 'train'
        if 'labels_test' in csv_path:
            self.mode = 'test'

    def __getitem__(self, idx):
        # load images and masks
        row = self.csv.loc[self.csv.image_id == idx]
        folder = row.folder.iloc[0]
        img_path = os.path.join(f'data/{folder}_Images/{folder}_Images', row.filename.iloc[0])
        img = np.array(Image.open(img_path).convert("RGB"))
        
        boxes = row[['xmin','ymin','xmax','ymax']].to_numpy()
        labels = [labelmap[i] for i in row['class'].tolist()]
        #labels = row['class'].tolist()

        target = {}
        
        target["image_id"] = row.image_id.iloc[0]

        if self.transforms is not None:
            r_number = random.randint(0,100)
            if self.mode == 'train' and r_number > 50:
                for i in range(5):
                    try:
                        
                        # Choose random box
                        r_number = random.randint(0,len(train_)-1)
                        r = train_.iloc[r_number]
                        img_pos = r[['xmin','ymin','xmax','ymax']].to_numpy()
                        fruit = Image.open(os.path.join(f'data/{r.folder}_Images/{r.folder}_Images', r.filename)).convert("RGB").crop(img_pos)

                        # Resize
                        random_size = random.randint(32,64)
                        x_size, y_size = fruit.size
                        ratio = y_size/x_size
                        x = random_size
                        y = x*ratio
                        fruit = fruit.resize((int(x),int(y)),Image.ANTIALIAS)
                        x_size, y_size = fruit.size
                        fruit = np.array(fruit)

                        # Make ellipse
                        image = np.zeros((y_size,x_size))
                        img_center = (x_size//2,y_size//2)
                        image = cv2.ellipse(image, img_center, img_center, 0, 0, 360, (255,255,255),-1)

                        # Choose random position
                        xmin, ymin = random.randint(0,512-x_size), random.randint(0,512-y_size)
                        xmax, ymax = xmin+x_size, ymin+y_size

                        # background + fruit
                        back = img[ymin:ymax, xmin:xmax].copy()
                        fruit[image==0] = back[image==0]
                        img[ymin:ymax, xmin:xmax] = fruit

                        boxes = np.append(boxes,np.array([[xmin,ymin,xmax,ymax]]),axis = 0)
                        labels.append(labelmap[r['class']])
                    except:
                        print('except')
                        pass
                
            annotations = {'image': img, 'bboxes':  boxes, 'category_id': labels}
            augmented = self.transforms(**annotations)
            img = augmented['image']
            boxes = torch.Tensor(augmented['bboxes'])
            labels = augmented['category_id']
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.tensor(labels)
        target["boxes"] = torch.tensor(boxes)
        target['img_scale'] = 1
        target['img_size'] = 512
        return img/255, target

    def __len__(self):
        return max(self.csv.image_id)
    
def vis(img, target):
    visualize.show_labeled_image(img.permute(1,2,0).numpy(), target['boxes'].numpy(), [labelmap_inv[i] for i in target["labels"].tolist()])

In [5]:
def get_net(model_name):
    if model_name == 'resnet101':
        backbone = torch.nn.Sequential(*(list(torchvision.models.resnet101(pretrained=True).children())[:-2]))
        backbone.out_channels = 2048
        
    if model_name == 'resnet152':
        backbone = torch.nn.Sequential(*(list(torchvision.models.resnet152(pretrained=True).children())[:-2]))
        backbone.out_channels = 1024
        
    elif model_name == 'densenet121':
        backbone = torchvision.models.densenet121(pretrained=True).features
        backbone.out_channels = 1024
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                                    output_size=7,
                                                    sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                       num_classes=4,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)
    return model

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.scaler = GradScaler()
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')
            
            t = time.time()
            summary_loss, cls_loss, box_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, cls_loss: {cls_loss.avg:.5f}, box_loss: {box_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss, cls_loss, box_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, cls_loss: {cls_loss.avg:.5f}, box_loss: {box_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    #os.remove(path)
                    pass

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.train()
        summary_loss = AverageMeter()
        cls_loss = AverageMeter()
        box_loss = AverageMeter()
        t = time.time()
        for step, (images, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'cls_loss: {cls_loss.avg:.5f}, ' + \
                        f'box_loss: {box_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).long() for target in targets]
                
                output = self.model(images, [{'boxes':boxes[i], 'labels':labels[i]} for i in range(len(labels))])
                
                loss = sum(loss for loss in output.values())
                loss_cls = output['loss_classifier']
                loss_box = output['loss_box_reg']
                
                summary_loss.update(loss.detach().item(), batch_size)
                cls_loss.update(loss_cls.detach().item(), batch_size)
                box_loss.update(loss_box.detach().item(), batch_size)

        return summary_loss, cls_loss, box_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        cls_loss = AverageMeter()
        box_loss = AverageMeter()
        t = time.time()
        for step, (images, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'cls_loss: {cls_loss.avg:.5f}, ' + \
                        f'box_loss: {box_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).long() for target in targets]

            self.optimizer.zero_grad()
            
            with autocast():
                output = self.model(images, [{'boxes':boxes[i], 'labels':labels[i]} for i in range(len(labels))])
            loss = sum(loss for loss in output.values())
            loss_cls = output['loss_classifier']
            loss_box = output['loss_box_reg']
            self.scaler.scale(loss).backward()

            summary_loss.update(loss.detach().item(), batch_size)
            cls_loss.update(loss_cls.detach().item(), batch_size)
            box_loss.update(loss_box.detach().item(), batch_size)

            self.scaler.step(self.optimizer)
            
            self.scaler.update()
            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss, cls_loss, box_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [6]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 2  # D5:2, D4: 3, D3: 4
    n_epochs = 2 # n_epochs = 40
    lr = 0.0001

    folder = 'logs/fasterrcnn-resnet101-pseudo-heavyaug-5add32_64-5ep-final'

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#    SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#    scheduler_params = dict(
#     max_lr=0.001,
#     epochs=n_epochs,
#     steps_per_epoch=int(2849 / batch_size),
#     pct_start=0.1,
#     anneal_strategy='cos', 
#     final_div_factor=10**5
#    )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_training():
    device = torch.device('cuda:0')
    net.to(device)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_ds),
        pin_memory=True,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )
    val_loader = torch.utils.data.DataLoader(
        val_ds, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(val_ds),
        pin_memory=False,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, val_loader)

In [8]:
if __name__ == '__main__':
    transform_heavy = Compose([
        RandomRotate90(p=0.5),
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
        
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.25),
        OneOf([
            CLAHE(),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),
        ], p=0.15),
        HueSaturationValue(p=0.25),
        #Affine(scale={'x':(1.2,1.5),'y':(1.2,1.5)}, p=0.15),
        ToTensorV2(),
    ], bbox_params=BboxParams(format='pascal_voc', min_area=0, min_visibility=0, label_fields=['category_id']))

    transform_test = Compose([
        #Normalize(),
        ToTensorV2(),
    ], bbox_params=BboxParams(format='pascal_voc', min_area=0, min_visibility=0, label_fields=['category_id']))
    
    train_ds = PassionFruitDataset(
        'data/labels_pseudo_train.csv',
        transform_heavy
    )

    val_ds = PassionFruitDataset(
        'data/labels_test.csv',
        transform_test
    )
    
    net = get_net('resnet101')
    run_training()